In [22]:

import logging
import os
import warnings

import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub

from datetime import datetime
from keras.preprocessing import image
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from tensorflow.keras import layers
from tensorflow.keras.models import Model
#../usr/lib/utils/utils.py
from utils.utils import *

warnings.filterwarnings('ignore')
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [23]:
defo_types = pd.read_csv("../input/planets-dataset/planet/planet/train_classes.csv")
defo_types.head()

In [24]:
label_freq = defo_types['tags'].apply(lambda x: str(x).split(' ')).explode().value_counts().sort_values(ascending=False)

In [25]:
# Bar plot
style.use("fivethirtyeight")
fig, ax = plt.subplots(figsize=(12,10))
sns.barplot(ax=ax, y=label_freq.index.values, x=label_freq, order=label_freq.index)
plt.title("Label frequency", fontsize=14)
plt.xlabel("")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax.bar_label(ax.containers[0])
plt.show()

In [26]:

rare = list(label_freq[label_freq<1000].index)
print(rare)


In [27]:
rep_dict = {'partly_cloudy': 'cloudy', 'haze':'cloudy'}
def lbl_reorg(x):
    res = []
    for i in str(x).split(" "):
        if i not in rare:
            if i not in rep_dict.keys():
                res.append(i)
            else:
                res.append(rep_dict[i])
    return res

In [28]:
defo_types['classes'] = defo_types['tags'].apply(lambda x:lbl_reorg(x))
defo_types.head()

In [29]:
X_train, X_val, y_train, y_val = train_test_split(defo_types['image_name'], defo_types['classes'], test_size=0.2, random_state=44)
print("Number of posters for training: ", len(X_train))
print("Number of posters for validation: ", len(X_val))

In [30]:
root = '../input/planets-dataset/planet/planet/train-jpg'
X_train = [os.path.join(root, str(f)+'.jpg') for f in X_train]
X_val = [os.path.join(root, str(f)+'.jpg') for f in X_val]
X_train[:3]

In [31]:
y_train = list(y_train)
y_val = list(y_val)
y_train[:3]

In [32]:
nobs = 8 # Maximum number of images to display
ncols = 4 # Number of columns in display
nrows = nobs//ncols # Number of rows in display

style.use("default")
plt.figure(figsize=(12,4*nrows))
for i in range(nrows*ncols):
    ax = plt.subplot(nrows, ncols, i+1)
    plt.imshow(Image.open(X_train[i]))
    plt.title(y_train[i], size=10)
    plt.axis('off')

In [33]:
# Fit the multi-label binarizer on the training set
print("Labels:")
mlb = MultiLabelBinarizer()
mlb.fit(y_train)

# Loop over all labels and show them
N_LABELS = len(mlb.classes_)
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i, label))

In [34]:
# transform the targets of the training and test sets
y_train_bin = mlb.transform(y_train)
y_val_bin = mlb.transform(y_val)

In [35]:
# Print example of movie posters and their binary targets
for i in range(3):
    print(X_train[i], y_train_bin[i])

In [36]:
IMG_SIZE = 128
CHANNELS = 3 
def parse_function(filename, label):
    """Function that returns a tuple of normalized image array and labels array.
    Args:
        filename: string representing path to image
        label: 0/1 one-dimensional array of size N_LABELS
    """
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    return image_normalized, label

In [37]:
BATCH_SIZE = 128
AUTOTUNE = tf.data.experimental.AUTOTUNE 
SHUFFLE_BUFFER_SIZE = 1000
N_LABELS = 8

In [38]:

def normalize(img, lbl):
    # one, zero = tf.ones_like(mask), tf.zeros_like(mask)
    img = img/255
    # mask = mask/tf.math.reduce_max(mask)
    # mask = tf.where(mask > 0.0, x=one, y=zero)
    return tf.cast(img, dtype=tf.float32), lbl

def rescale(img, lbl, scale=(128,128)):
    img = tf.image.resize(img, scale, preserve_aspect_ratio=True, antialias=True)
    mask = tf.image.resize(mask, scale, preserve_aspect_ratio=True, antialias=True)
    return tf.cast(img, dtype=tf.float32), lbl


def brightness(img1, mask):
    rnd = tf.random.uniform(shape=[], maxval=2, minval=0, dtype=tf.int32,seed=5)
    if rnd == 0:
        img1 = tf.image.adjust_brightness(img1, 0.1)
#     img2 = tf.image.adjust_brightness(img2, 0.1)
    return img1, mask

def gamma(img1, lbl):
    rnd = tf.random.uniform(shape=[], maxval=2, minval=0, dtype=tf.int32,seed=5)
    if rnd == 0:
        img1 = tf.image.adjust_gamma(img1, 0.1)
#     img2 = tf.image.adjust_gamma(img2, 0.1) 
    return img1, lbl

def hue(img1, lbl):
    rnd = tf.random.uniform(shape=[], maxval=2, minval=0, dtype=tf.int32,seed=5)
    if rnd == 0:
        img1 = tf.image.adjust_hue(img1, -0.1)
#     img2 = tf.image.adjust_hue(img2, -0.1)
    return img1, lbl

# def crop(img, mask):
#      img = tf.image.central_crop(img, 0.7)
#      img = tf.image.resize(img, (128,128))
#      mask = tf.image.central_crop(mask, 0.7)
#      mask = tf.image.resize(mask, (128,128))
#      mask = tf.cast(mask, tf.uint8)
#      return img, mask

def flip_hori(img1, lbl):
    rnd = tf.random.uniform(shape=[], maxval=2, minval=0, dtype=tf.int32,seed=5)
    if rnd == 0:
        img1 = tf.image.flip_left_right(img1)
#     img2 = tf.image.flip_left_right(img2)
#         mask = tf.image.flip_left_right(mask)
    return img1, lbl

def flip_vert(img1, lbl):
    rnd = tf.random.uniform(shape=[], maxval=2, minval=0, dtype=tf.int32,seed=5)
    if rnd == 0:
        img1 = tf.image.flip_up_down(img1)
#     img2 = tf.image.flip_up_down(img2)
#         mask = tf.image.flip_up_down(mask)
    return img1, lbl

def rotate(img1, lbl):
    rnd = tf.random.uniform(shape=[], maxval=2, minval=0, dtype=tf.int32,seed=5)
    if rnd == 0:
        img1 = tf.image.rot90(img1)
#     img2 = tf.image.rot90(img2)
#         mask = tf.image.rot90(mask)
    return img1, lbl


In [39]:
def create_dataset(filenames, labels, is_training=True):
    """Load and parse dataset.
    Args:
        filenames: list of image paths
        labels: numpy array of shape (BATCH_SIZE, N_LABELS)
        is_training: boolean to indicate training mode
    """
    
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    
    if is_training == True:
        # This is a small dataset, only load it once, and keep it in memory.
        #dataset = dataset.cache()
        # Shuffle the data each buffer size
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
        
    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [40]:
train_ds = create_dataset(X_train, y_train_bin)
val_ds = create_dataset(X_val, y_val_bin)

In [41]:
train_ds.element_spec

In [42]:
# perform augmentation on train data only
# train_ds = train_ds.map(normalize)
# train_ds = train_ds.map(rescale)
train_ds = train_ds.map(brightness)
train_ds = train_ds.map(gamma)
train_ds = train_ds.map(flip_hori)
train_ds = train_ds.map(flip_vert)
train_ds = train_ds.map(rotate)

In [43]:
for f, l in train_ds.take(1):
    print("Shape of features array:", f.numpy().shape)
    print("Shape of labels array:", l.numpy().shape)

In [44]:
# feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
# feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
#                                          input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))
feat_model = tf.keras.applications.MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))
# feat_model.summary()

In [45]:
def create_model(img_size,channels):
    feat_model = tf.keras.applications.MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(img_size,img_size,channels))
    
    for layer in feat_model.layers:
        layer.trainable = False
        
    x = feat_model.output
    x = layers.GlobalAveragePooling2D()(x)
#     x = layers.Dense(1024, activation='relu', name='last_linear')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(N_LABELS, activation='sigmoid', name='output')(x)
    model = Model(inputs=feat_model.input, outputs=x, name='multi_class_multi_label_classifier')
    return model
# feature_extractor_layer.trainable = False
# model = tf.keras.Sequential([
#     feature_extractor_layer,
#     layers.Dense(1024, activation='relu', name='hidden_layer'),
#     layers.Dense(N_LABELS, activation='sigmoid', name='output')
# ])

# model.summary()
model = create_model(IMG_SIZE,CHANNELS)

In [46]:
model.summary()

In [47]:
for batch in train_ds:
#     print(batch[0].shape)
    print(model.predict(batch[0])[:1])
    break

In [48]:
@tf.function
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

In [49]:
@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [51]:
LR = 1e-4
EPOCHS = 5
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
  loss=macro_soft_f1,
  metrics=[macro_f1])

In [52]:
start = time()
history = model.fit(train_ds,
                    epochs=EPOCHS,
                    validation_data=create_dataset(X_val, y_val_bin))
print('\nTraining took {}'.format(print_time(time()-start)))

In [53]:
losses, val_losses, macro_f1s, val_macro_f1s = learning_curves(history)

In [54]:
print("Macro soft-F1 loss: %.2f" %val_losses[-1])
print("Macro F1-score: %.2f" %val_macro_f1s[-1])

In [55]:
model_bce = create_model(IMG_SIZE,CHANNELS)

model_bce.compile(
    optimizer=tf.keras.optimizers.Adam(lr=5e-4),
    loss=tf.keras.metrics.binary_crossentropy,
    metrics=[macro_f1])
    
start = time()
history_bce = model_bce.fit(train_ds,
                            epochs=EPOCHS,
                            validation_data=create_dataset(X_val, y_val_bin))
print('\nTraining took {}'.format(print_time(time()-start)))

In [56]:
model_bce_losses, model_bce_val_losses, model_bce_macro_f1s, model_bce_val_macro_f1s = learning_curves(history_bce)

In [57]:
print("Macro soft-F1 loss: %.2f" %model_bce_val_losses[-1])
print("Macro F1-score: %.2f" %model_bce_val_macro_f1s[-1])

In [58]:
# Get all label names
label_names = mlb.classes_
# Performance table with the first model (macro soft-f1 loss)
grid = perf_grid(val_ds, y_val_bin, label_names, model)
# Performance table with the second model (binary cross-entropy loss)
grid_bce = perf_grid(val_ds, y_val_bin, label_names, model_bce)

In [59]:
grid_bce.head()

In [60]:
# Get the maximum F1-score for each label when using the second model and varying the threshold
max_perf = grid_bce.groupby(['id', 'label', 'freq'])[['f1']].max().sort_values('f1', ascending=False).reset_index()
max_perf.rename(columns={'f1':'f1max_bce'}, inplace=True)
max_perf.style.background_gradient(subset=['freq', 'f1max_bce'], cmap=sns.light_palette("lightgreen", as_cmap=True))

In [61]:
print("Correlation between label frequency and optimal F1 with bce: %.2f" %max_perf['freq'].corr(max_perf['f1max_bce']))

In [63]:
top5 = max_perf.head(5)['id']

In [64]:
style.use("default")
for l in top5:
    
    label_grid = grid.loc[grid['id']==l, ['precision','recall','f1']]
    label_grid = label_grid.reset_index().drop('index', axis=1)
    
    label_grid_bce = grid_bce.loc[grid_bce['id']==l, ['precision','recall','f1']]
    label_grid_bce = label_grid_bce.reset_index().drop('index', axis=1)
    
    plt.figure(figsize=(9,3))

    ax = plt.subplot(1, 2, 1)
    plt.xticks(ticks=np.arange(0,110,10), labels=np.arange(0,110,10)/100, fontsize=10)
    plt.yticks(fontsize=8)
    plt.title('Performance curves - Label '+str(l)+' ('+label_names[l]+')\nMacro Soft-F1', fontsize=10)
    label_grid.plot(ax=ax)
    
    ax = plt.subplot(1, 2, 2)
    plt.xticks(ticks=np.arange(0,110,10), labels=np.arange(0,110,10)/100, fontsize=8)
    plt.yticks(fontsize=8)
    plt.title('Performance curves - Label '+str(l)+' ('+label_names[l]+')\nBCE', fontsize=10)
    label_grid_bce.plot(ax=ax)
    
    plt.tight_layout()
    plt.show()

In [66]:
# Predict on the validation set with both models
y_hat_val = model.predict(val_ds)
y_hat_val_bce = model_bce.predict(val_ds)

In [67]:
style.use("default")
for l in top5:
        
    plt.figure(figsize=(9,3))
    
    ax = plt.subplot(1, 2, 1)
    plt.xticks(ticks=np.arange(0,1.1,0.1), fontsize=8)
    plt.yticks(fontsize=8)
    plt.title('Probability distribution - Label '+str(l)+' ('+label_names[l]+')\nMacro Soft-F1', fontsize=10)
    plt.xlim(0,1)
    ax = sns.distplot(y_hat_val[:,l], bins=30, kde=True, color="g")
    
    ax = plt.subplot(1, 2, 2)
    plt.xticks(ticks=np.arange(0,1.1,0.1), fontsize=8)
    plt.yticks(fontsize=8)
    plt.title('Probability distribution - Label '+str(l)+' ('+label_names[l]+')\nBCE', fontsize=10)
    plt.xlim(0,1)
    ax = sns.distplot(y_hat_val_bce[:,l], bins=30, kde=True, color="b")
    
    plt.tight_layout()
    plt.show()

In [68]:
def show_prediction(title, movies_df, model):
    
    # Get movie info
    imdbId = movies.loc[movies['Title']==title]['imdbId'].iloc[0]
    genre = movies.loc[movies['Title']==title]['Genre'].iloc[0]
    img_path = os.path.join('./data/movie_poster/images', str(imdbId)+'.jpg')

    # Read and prepare image
    img = image.load_img(img_path, target_size=(IMG_SIZE,IMG_SIZE,CHANNELS))
    img = image.img_to_array(img)
    img = img/255
    img = np.expand_dims(img, axis=0)

    # Generate prediction
    prediction = (model.predict(img) > 0.5).astype('int')
    prediction = pd.Series(prediction[0])
    prediction.index = mlb.classes_
    prediction = prediction[prediction==1].index.values

    # Dispaly image with prediction
    style.use('default')
    plt.figure(figsize=(8,4))
    plt.imshow(Image.open(img_path))
    plt.title('\n\n{}\n\nGenre\n{}\n\nPrediction\n{}\n'.format(title, genre, list(prediction)), fontsize=9)
    plt.show()

In [69]:

#t = datetime.now().strftime("%Y%m%d_%H%M%S")
#export_path = "./models/soft-f1_{}".format(t)
model.save_weights('defo_ks2.h5')
#print("Model with macro soft-f1 was exported in this path: '{}'".format(export_path))

# export_path_bce = "./models/bce_{}".format(t)
# tf.keras.experimental.export_saved_model(model_bce, export_path_bce)
# print("Model with bce was exported in this path: '{}'".format(export_path_bce))

In [ ]:
X_train[0]

Load the model and Inference

In [ ]:
from tensorflow.keras.models import load_model
import cv2

In [ ]:
inf_model = create_model(IMG_SIZE,CHANNELS)
inf_model.load_weights('./defo_ks1.h5')

In [ ]:
img002 = cv2.imread(X_train[200])
img002 = cv2.resize(img002, (224, 224), interpolation = cv2.INTER_NEAREST )
print(img002.shape)

In [ ]:
plt.imshow(img002)

In [ ]:
input_img = img002[np.newaxis, ...]/255
res = inf_model(input_img)

In [ ]:
np.argmax(res[0])

In [ ]:
res

create and download sample dataset

In [ ]:
resf =  model(input_img/255)
resf

In [ ]:
sample_ds = []
for i in range(5, 105):
    img = cv2.imread(X_train[i])
    img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_NEAREST )/255
    img = img[np.newaxis, ...]
    sample_ds.append(img)
    
sample_dt = np.vstack(sample_ds)
    

In [ ]:
sample_dt.shape

In [ ]:
np.savez_compressed('./sample_RGB_dataset.npz', images=sample_dt)

In [ ]:
new_ds.shape